In [35]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [36]:
# Function to scrape Amazon reviews
def scrape_amazon_reviews(asin, max_pages=5):
    base_url = f"https://{MARKETPLACE}/product-reviews/{asin}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    reviews = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}?pageNumber={page}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve page {page}. HTTP Status Code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")
        review_blocks = soup.find_all("div", {"data-hook": "review"})

        for review in review_blocks:
            try:
                reviewer_id = review.get("id", "N/A")
                reviewer_name = review.find("span", {"class": "a-profile-name"}).text.strip()
                helpful = review.find("span", {"data-hook": "helpful-vote-statement"})
                helpful = int(helpful.text.split()[0]) if helpful else 0
                overall = float(review.find("i", {"data-hook": "review-star-rating"}).text.split()[0])
                summary = review.find("a", {"data-hook": "review-title"}).text.strip()
                unix_review_time = int(time.time())  # Simulate a Unix time
                review_time = review.find("span", {"data-hook": "review-date"}).text.strip()
                day_diff = 0  # Placeholder: calculate difference from review_time if needed
                helpful_yes = helpful
                total_vote = helpful_yes  # Simplified: Adjust based on review format

                reviews.append([
                    reviewer_id,
                    asin,
                    reviewer_name,
                    helpful,
                    overall,
                    summary,
                    unix_review_time,
                    review_time,
                    day_diff,
                    helpful_yes,
                    total_vote,
                ])
            except Exception as e:
                print(f"Error parsing review: {e}")
        time.sleep(1)  # Be respectful with delay

    return reviews # This line was likely misaligned. Make sure it's indented correctly.

In [37]:
# Function to save data to CSV
def save_to_csv(data, filename="amazon_reviews.csv"):
   column_names = [
       "Reviewer ID",
       "Asin",
       "ReviewerName",
       "Helpful",
       "overall",
       "summary",
       "unixReviewTime",
       "reviewTime",
       "day_diff",
       "helpful_yes",
       "total_vote",
   ]
   with open(filename, "w", newline="", encoding="utf-8") as file:
       writer = csv.writer(file)
       writer.writerow(column_names)
       writer.writerows(data)
   print(f"Data saved to {filename}")


In [38]:
# Example usage
if __name__ == "__main__":
   MARKETPLACE = "www.amazon.com"
   ASIN = "B08N5WRWNW"  # Example ASIN (replace with your own)

   reviews = scrape_amazon_reviews(ASIN, max_pages=3)
   save_to_csv(reviews)

Failed to retrieve page 1. HTTP Status Code: 404
Data saved to amazon_reviews.csv
